In [ ]:
# !pip install mp-api

In [1]:
import pandas as pd
import tqdm

In [2]:
from mp_api.client import MPRester

# あなたのMaterials ProjectのAPIキーをここに入力してください
YOUR_API_KEY = "RnqyA0tiXmBuAFbMEAIJM2n5AUaX8ZP7"

In [3]:
mpr = MPRester(YOUR_API_KEY)

In [ ]:
# sulfur_data = mpr.query(criteria={"elements" : {"$in" : ["S", "-O"]}, "nelements" : {"$gt" : 2}}, properties=["e_above_hull", "band_gap", "cif"])
# oxide_data = mpr.query(criteria={"elements" : {"$in" : ["-S", "O"]}, "nelements" : {"$gt" : 2}}, properties=["e_above_hull", "band_gap", "cif"])

# MPResterオブジェクトを初期化
with MPRester(YOUR_API_KEY) as mpr:
    data = mpr.materials.summary.search(
        fields=[
            'material_id',
            "formula_pretty",
            "band_gap",
            "formation_energy_per_atom",
            "energy_above_hull",
            "structure",
                  ]
    )

In [ ]:
# # 特定の材料IDに対応する結晶構造を取得します。
# material_id = "mp-757220"  # 例としての材料ID formula_pretty LuH6(ClO5)3
# structure = mpr.get_structure_by_material_id(material_id)

# # CIFファイルとして保存します。
# structure.to(filename=f"{material_id}.cif")

In [ ]:
# dataからmaterial_idの値のみを抽出
mpids = [d.material_id for d in data]
mpids

In [ ]:
# IDのみを抽出する
extracted_ids = [mpid.split("(")[-1].split(")")[0] for mpid in mpids]
extracted_ids

In [ ]:
# extracted_idsのmp-以降を抽出する
extracted_ids_numbers = [mpid.split("-")[-1] for mpid in extracted_ids]
extracted_ids_numbers

In [ ]:
# dataからformula_prettyの値のみを抽出
formulas = [d.formula_pretty for d in data]
formulas


In [ ]:
# dataからband_gapの値のみを抽出
band_gaps = [d.band_gap for d in data]
band_gaps

In [ ]:
# dataからformation_energy_per_atomの値のみを抽出
formation_energies = [d.formation_energy_per_atom for d in data]
formation_energies

In [ ]:
# dataからenergy_above_hullの値のみを抽出
energy_above_hulls = [d.energy_above_hull for d in data]
energy_above_hulls

In [ ]:
# formulas, ids, band_gaps, formation_energies, energy_above_hullsをpandasのDataFrameに変換
df_ = pd.DataFrame({
    "formula": formulas,
    "id": extracted_ids,
    "band_gap": band_gaps,
    "formation_energy": formation_energies,
    "energy_above_hull": energy_above_hulls,
    # "structure": structures,
})
df_

In [ ]:
len(df_)

In [ ]:
#df_をはじめの1000行だけ取得
df_1000 = df_.head(1000)
df_1000

In [4]:
# /workdir/practical-mi-guide/chapter3/input/material_projects_api_cif.csvを読み込み
df_cif = pd.read_csv("/workdir/practical-mi-guide/chapter3/input/material_projects_api_cif.csv")
df_cif

,formula,id,band_gap,formation_energy,energy_above_hull
0,Cs3Nd(MoO4)3,mp-573369,3.7419,-2.414818,0.000000
1,Ba(PdS2)2,mp-28967,0.7792,-1.124079,0.000000
2,Na7Ta3Zn(SiO4)6,mp-2712904,2.5286,-2.912354,0.049678
3,TiZnBiO5,mp-1042447,0.0000,-2.236384,0.077777
4,Na2YNb3Si4(SO12)2,mp-2715697,1.8105,-2.837049,0.079832
...,...,...,...,...,...
153230,Na2ScAgF6,mp-1111316,1.7862,-2.958455,0.167160
153231,KP2Au5S8,mp-561218,1.4084,-0.560986,0.000000
153232,AlVF5,mp-1248567,3.1768,-3.481125,0.022430
153233,RbSrMg6,mp-1017292,0.0000,0.222729,0.251729


In [5]:
extracted_ids = df_cif["id"].values
extracted_ids

array(['mp-573369', 'mp-28967', 'mp-2712904', ..., 'mp-1248567',
       'mp-1017292', 'mp-583712'], dtype=object)

In [6]:
len(extracted_ids)

153235

In [8]:
formulas = df_cif["formula"].values
formulas

array(['Cs3Nd(MoO4)3', 'Ba(PdS2)2', 'Na7Ta3Zn(SiO4)6', ..., 'AlVF5',
       'RbSrMg6', 'Si3N4'], dtype=object)

In [10]:
import os

# mids=mpr.materials.search(material_ids= material_id_list[0], fields=["structure"])

output_dir = "/workdir/practical-mi-guide/chapter3/input/material_projects_api_cif"

for i in range(len(df_cif)):
    material_id_list_tmp = extracted_ids[i]
    formulas_tmp = formulas[i]
    search_results = mpr.materials.search(material_ids=material_id_list_tmp, fields=["structure"])
    
    # search_results はリストと仮定しています。
    # もし検索結果がリストでない場合、この部分を適切に修正する必要があります。
    for result in search_results:
        structure = result.structure
        filename = f"/workdir/practical-mi-guide/chapter3/input/material_projects_api_cif/cif_{material_id_list_tmp}_{formulas_tmp}.cif"
        structure.to(filename=filename, fmt='cif')

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # /home/takahara/app_example/output/material_projects_api_cif/cif_mp-766094_NbO2F.cifを読み込んでpymatgenのstrctureオブジェクトにする
# from pymatgen.core.structure import Structure

# structure = Structure.from_file("/home/takahara/app_example/output/material_projects_api_cif/cif_mp-766094_NbO2F.cif")
# structure

In [ ]:
# home/takahara/app_example/output/material_projects_api_cifのファイルの数を確認
import os

path = "/home/takahara/app_example/output/material_projects_api_cif_1000"
files = os.listdir(path)
files

In [ ]:
len(files)

In [ ]:
# df_1000を/home/takahara/app_example/output/material_projects_api_cif_1000.csvとして保存
df_1000.to_csv("/home/takahara/app_example/output/material_projects_api_cif_1000/material_projects_api_cif_1000.csv", index=False)

In [ ]:
# df_をcsvファイルとして保存
df_.to_csv("/home/takahara/app_example/output/material_projects_api_cif_1000/material_projects_api_cif.csv", index=False)

In [ ]:
'''
material_id
band_gap
fields_not_requested
[
'builder_meta', 'nsites', 'elements', 'nelements',
'composition', 'composition_reduced', 'formula_pretty',
'formula_anonymous', 'chemsys', 'volume', 'density',
'density_atomic', 'symmetry', 'property_name',
'deprecated', 'deprecation_reasons', 'last_updated',
'origins', 'warnings', 'structure', 'task_ids',
'uncorrected_energy_per_atom', 'energy_per_atom',
'formation_energy_per_atom',
'energy_above_hull', 'is_stable',
'equilibrium_reaction_energy_per_atom',
'decomposes_to', 'xas', 'grain_boundaries',
'cbm', 'vbm', 'efermi', 'is_gap_direct',
'is_metal', 'es_source_calc_id',
'bandstructure', 'dos', 'dos_energy_up',
'dos_energy_down', 'is_magnetic', 'ordering',
'total_magnetization', 'total_magnetization_normalized_vol',
'total_magnetization_normalized_formula_units', 'num_magnetic_sites',
'num_unique_magnetic_sites', 'types_of_magnetic_species',
'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh',
'universal_anisotropy', 'homogeneous_poisson',
'e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max',
'weighted_surface_energy_EV_PER_ANG2', 'weighted_surface_energy', 'weighted_work_function',
'surface_anisotropy', 'shape_factor', 'has_reconstructed',
'possible_species', 'has_props', 'theoretical', 'database_IDs'
]
'''